<a href="https://colab.research.google.com/github/ningkko/Biomedinfo/blob/master/BMI702Assgn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authored by Yasha Ektefaie and Mert Eden

**Do not change the runtime environment from GPU to standard runtime. You'll need GPU for Q3. **

***Please make a copy of this Colab notebook so that you can save your codes!***

## Fairness:

Code blocks titled 2.1 and 2.2 pertain to questions 2.1 and 2.2 respectively on the homework. Below are some blocks to set you up for training the logistic regression model. It will download the dataset and do some preproccessing. For this exercise *you will not be training the model yourself*. 

In [1]:
!wget -O credit.csv https://www.openml.org/data/get_csv/31/dataset_31_credit-g.csv

--2022-03-14 00:57:15--  https://www.openml.org/data/get_csv/31/dataset_31_credit-g.csv
Resolving www.openml.org (www.openml.org)... 131.155.11.11
Connecting to www.openml.org (www.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘credit.csv’

credit.csv              [ <=>                ] 148.44K  --.-KB/s    in 0.03s   

2022-03-14 00:57:15 (5.20 MB/s) - ‘credit.csv’ saved [152006]



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('credit.csv')

In [4]:
remove = ['installment_commitment', 'other_parties','housing', 'property_magnitude',
          'other_payment_plans','existing_credits', 'residence_since']
  
cols_keep = [i for i in df.columns if i not in remove]

In [5]:
#checking status different levels of credit
checking_status_mappings = {"'no checking'": 0, "'<0'": 1, "'0<=X<200'":2, "'>=200'":3}

#credit problem: have they paid (0) if not paid then problem 1
credit_status_mappings = {"'no credits/all paid'":0, "'all paid'":0,
                          "'existing paid'":0,
                          "'critical/other existing credit'":1,
                          "'delayed previously'":1 }

#credit pupose: car is 0, education is 1, business is 2, objects: 3, other: 4
credit_purpose_mapping = {
    "radio/tv": 3,
    "education": 1,
    "furniture/equipment":3,
    "'new car'": 0 ,
    "'used car'":0, 
    "business": 2, 
    "'domestic appliance'": 3,
    "repairs": 3,
    "other": 4, 
    "retraining": 4
    }

#employment: unemployed is 0, <1 is 1, 1<=X<4 is 2, 4<=X<7 is 3, >= 7 is 4
employment_mapping = {
    "'>=7'": 4, 
    "'1<=X<4'": 2, 
    "'4<=X<7'": 3, 
    'unemployed': 0 , 
    "'<1'": 1
}

#saving status: no savings is 0, <100 is 1, 100-500 is 2, 500-1000 is 3, >1000 4
savings_status = {"'no known savings'":0, 
"'<100'":1, 
"'500<=X<1000'":3, 
"'>=1000'":4,
"'100<=X<500'": 2}

#job binning just provide number to each option
job_mapping = {
    'skilled': 0, 
    "'unskilled resident'": 1, 
    "'high qualif/self emp/mgmt'": 2,
    "'unemp/unskilled non res'": 3}

In [6]:
trunc_df = df[cols_keep].copy()
trunc_df = trunc_df.replace({
                  "checking_status": checking_status_mappings,
                  "purpose": credit_purpose_mapping, 
                  "credit_history": credit_status_mappings,
                  "employment": employment_mapping,
                  "job": job_mapping,
                  "savings_status":savings_status})

#Gender 1 if female, 0 if male
trunc_df['sex'] = np.where(trunc_df['personal_status'].str.contains('female'), 1, 0)
#Telephone 1 if have telephone, 0 otherwise
trunc_df['own_telephone'] = np.where(trunc_df['own_telephone'].str.contains('none'), 0, 1)
#Foreign worker 1 if foreign worker, 0 otherwise
trunc_df['foreign_worker'] = np.where(trunc_df['foreign_worker'].str.contains('yes'), 1, 0)
#Class 1 if good, 0 if bad
trunc_df['class'] = np.where(trunc_df['class'].str.contains('good'), 1, 0)
del trunc_df['personal_status']
trunc_df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,age,job,num_dependents,own_telephone,foreign_worker,class,sex
0,1,6,1,3,1169,0,4,67,0,1,1,1,1,0
1,2,48,0,3,5951,1,2,22,0,1,0,1,0,1
2,0,12,1,1,2096,1,3,49,1,2,0,1,1,0
3,1,42,0,3,7882,1,3,45,0,2,0,1,1,0
4,1,24,1,0,4870,1,2,53,0,2,0,1,0,0


In [7]:
X_train, x_valid, y_train, y_valid = train_test_split(trunc_df.drop('class',axis=1), 
                                                    trunc_df['class'], test_size=0.30, 
                                                    random_state=101)


In [8]:
biased_lr = LogisticRegression(max_iter = 1000)
biased_lr.fit(X_train,y_train)
print(f"Logistic regression valid accuracy: {biased_lr.score(x_valid, y_valid)}")

Logistic regression valid accuracy: 0.7033333333333334


### 2.1 Compute the demographic parity with respect to the sensitive trait `sex`.

Define a function parity that takes the predictions on the test set of our
linear regressor, and a column of sensitive attributes and computes the difference in demographic parity. 

Run parity on the sensitive attribute `sex` and briefly discuss whether or not this linear regressor satisfies demographic parity.

In [9]:
#Here we are comparing the rates of good credit among males versus females

def parity(pred, sens):
  return 0

sens = x_valid['sex']
pred = biased_lr.predict(x_valid)

parity(pred, sens)

0

### 2.2 Compute equal opportunity with respect to the sensitive trait `sex`.
Define a function equal_opportunity that takes the predictions on the test set of our linear regressor, a column of ground truth labels, and a column of sensitive attributes and computes the difference in equal opportunity.

Compute equal opportunity with respect to the sensitive trait `sex`. How can we interpret the calculated rate of equal opportunity? What is the meaning of this rate for someone looking to get a loan from the bank?

In [10]:
def equal_opportunity(pred, labels, sens):
  return 0
sens = x_valid['sex']
labels = y_valid
pred = biased_lr.predict(x_valid)
equal_opportunity(pred, labels, sens)

0

# 3 Explainibility.

**Before you start working on Q3, in case you have changed your runtime, check if you still have GPU resource by clicking on Runtime > Change runtime type. "Hardware acceleartor" should be set as "GPU". **

For the next problem you will be running integrated gradients on a binary classifier trained on the PCam dataset. PCam or PatchCamelyon is a histopathological dataset of lymph nodes. The classifier is trained to detect whether or not there is any metastatis present in these lymph nodes. Below is some starter code to download required packages, install the dataset and train the model. Code blocks are titled with the respective excerise number in the assignment.

## Setting up
We'll be using pyTorch to train our CNN. **All the NN training is already implemented so you do not have to understand how it works.** Please just change the runtime type as explained above and run the following lines of code one at a time. 

In [11]:
# Getting pytorch
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install captum
!pip install flask_compress
!pip install --upgrade --no-cache-dir gdown

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.3 MB 25 kB/s 
     |████████████████████████████████| 12.7 MB 44.2 MB/s 
     |████████████████████████████████| 7.6 MB 10.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torc

     |████████████████████████████████| 1.4 MB 11.9 MB/s 
     |████████████████████████████████| 357 kB 11.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=7d58a75e12998cc613b08f076e98ec40adaa2bc1827abbfdb28afb0cd4796b62
  Stored in directory: /tmp/pip-ephem-wheel-cache-hjq0agbi/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [12]:
!gdown --id 1Ka0XfEMiwgCYPdTI-vv6eUElOBnKFKQ2
!gdown --id 1269yhu3pZDP8UYFQs-NYs3FPwuK-nGSG

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Ka0XfEMiwgCYPdTI-vv6eUElOBnKFKQ2
To: /content/camelyonpatch_level_2_split_train_x.h5.gz
100% 6.42G/6.42G [01:11<00:00, 89.7MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1269yhu3pZDP8UYFQs-NYs3FPwuK-nGSG
To: /content/camelyonpatch_level_2_split_train_y.h5.gz
100% 21.4k/21.4k [00:00<00:00, 27.4MB/s]


In [ ]:
!gzip -d camelyonpatch_level_2_split_train_x.h5.gz
!gzip -d /content/camelyonpatch_level_2_split_train_y.h5.gz

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir -p /content/gdrive/My\ Drive/BMI702/ # You can change directory under 'My Drive' as you want
!cp camelyonpatch_level_2_split_train_x.h5 /content/gdrive/My\ Drive/BMI702/

# Code to Setup Dataset

In [ ]:
PER_TRAIN = 50 / 100
BATCH_SIZE = 64

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from torch import optim
from torch.autograd import Variable 
from torch.utils import data
from PIL import Image
from matplotlib.pyplot import imshow
import h5py 
import math
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0)


class PCam(torch.utils.data.Dataset):
  def __init__(self, transform=None):
      super(PCam, self).__init__()
      self.images = h5py.File('/content/camelyonpatch_level_2_split_train_x.h5', 'r')
      self.labels = h5py.File('/content/camelyonpatch_level_2_split_train_y.h5', 'r')
      self.transform=transform

  def __getitem__(self, index): 
    image = torch.tensor(self.images['x'][index]).float()
    image = (image / 255.0).permute(2,1,0)
    if self.transform:
      image = self.transform(image, self.labels['y'][index])

    return (image, Variable(torch.tensor(self.labels['y'][index]).reshape(-1).float()))
  def __len__(self):
    return len(self.images['x'])



In [ ]:
def initial_transform(tensor, label):
  transform = torchvision.transforms.Compose([
      torchvision.transforms.ToPILImage(), # Convert np array to PILImage
      
      # Resize image to 224 x 224 as required by most vision models
      torchvision.transforms.Resize(
          size=(224, 224)
      ),
      
      # Convert PIL image to tensor with image values in [0, 1]
      torchvision.transforms.ToTensor(),
  ])
  return transform(tensor)


pcam = PCam(transform=initial_transform)

sz = len(pcam)
size_train = math.floor(sz - (sz * (1 - PER_TRAIN)))
size_test = sz - size_train

print(f"Training size: {size_train}")
print(f"Testing size: {size_test}")


train_split, test_split = torch.utils.data.random_split(pcam, [size_train, size_test])
train_set = DataLoader(train_split, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=1)
test_set = DataLoader(test_split, batch_size=1,
                        shuffle=True, num_workers=1)

## Take a peek of our data
Our transformed PCAM dataset consists of 262,144 data each consists of a 224*224 image with 3 color channels and 1 label to annotate if the image contains metastatic tissue (1) or not (0).

In [ ]:
image, label = pcam[0]
print("Image of shape ", image.shape)
print(image)
print("Label: ", label)

"""
If everything ran correctly should see:Image of shape  torch.Size([3, 224, 224])
And then should see a large printed out 2D Array with text like:

tensor([[[0.8863, 0.8863, 0.8784,  ..., 0.3451, 0.3373, 0.3333],

printed out.
"""

In [ ]:
"""
*********************
*********************
!!!!!PLEASE READ!!!!!
*********************
*********************

This is the code to train the model. 
If everything above ran correctly upon running this block you should see
lines like: 

[1,     1] loss: 0.00021
Accuracy: 0.010625000111758709

Make sure you switched the runtime to GPU as described in the beginning.
Even with the GPU this block of code will take around 15-30 minutes to run. 
As long as lines like the one shown above are being printed and the accuracy
is increasing everything is fine. Just don't loose internet connect or let your
computer fall asleep during training. 

It is important you run this block of code once. Do not rerun it as you will
rerun training. This is why after this block of code there is code to SAVE
your model and LOAD the model. That way you can run training and save the model
once and then load as needed.

The way all this works is we explicitly create a folder in your google drive 
where we are storing the training data and saved model. Once the training data
and model is saved, even if your runtime disconnects or you close this tab
the model and data is still saved to your google drive. All you have to do 
to start up again is remount to that directory and load in the model. 

Run these next cells carefully.

"""
import torchvision.models as models
from tqdm.notebook import tqdm
import gc 
import copy 

gc.collect()
torch.cuda.empty_cache()

MY_MODEL = models.resnet18() 

"""
If you find training is taking too long consider changing epoch here from 2 to 1
So instead of epochs = 2, have it be epochs = 1

"""

def train(dataset, epochs=2):
  model = copy.deepcopy(MY_MODEL)

  model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 1))

  error = nn.BCEWithLogitsLoss()
  learning_rate = 0.001
  optimizer = torch.optim.Adam(model.parameters())

  running_loss = 0.0
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model.train()
  model = model.to(device)
  for epoch in range(epochs):
    correct = 0
    for i, (train, labels) in enumerate(dataset):
            # Clear gradients
            train = train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            if i == 0:
              print("here")
            # Forward propagation
            outputs = model(train)        
            # Compute Loss
            loss = error(outputs, labels.view(-1, 1).float())
            correct += labels.eq(torch.round(torch.sigmoid(outputs))).sum()
            # Calculating gradients
            loss.backward()        
            # Update parameters
            optimizer.step()   
            # print statistics
            running_loss += loss.item()
            if i % 50 == 0:    # print every 50 mini-batches
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss / (BATCH_SIZE * 50)))
                running_loss = 0.0
                print(f"Accuracy: {correct / (BATCH_SIZE * 50)}")
                correct = 0
  return model

#model = train(train_set)

In [ ]:
#This is the code to train the model
model = train(train_set)

You can save the model to your drive and load it in the future.

In [ ]:
!mkdir -p /content/gdrive/My\ Drive/BMI702/ # You can change the directory under "My Drive" as you want.

In [ ]:
#Code to save your model
model_save_name = 'pcam_model'
path = f"/content/gdrive/My Drive/BMI702/{model_save_name}" 
torch.save(model, path)

In [ ]:
#Code to load the model
#If your collab notebook instance crashes you can reload the model by running these lines
#We first remount to the google drive folder were everything is stored
#Then we just pull from the model file stored in that folder

from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'pcam_model'
path = f"/content/gdrive/My Drive/BMI702/{model_save_name}" 
model = torch.load(path)

### Evaluate how our model did on the first 2000 testing points

In [ ]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
correct = 0
total = 0
with torch.no_grad():
    for i, (train, labels) in enumerate(test_set):
      if i > 2000:
        break
      images = train.to(device)
      labels = labels.to(device)
      outputs = model(images)
      correct += labels.view(-1,1).eq(torch.round(torch.sigmoid(outputs))).sum()
      total += 1
      

print('Accuracy of the network on the 2000 test images: %d %%' % (100 * (correct / total)))

### Let's visualize some slides.

Although we're no pathologists, we do get some valuable insight as to what our data is by looking at it. Below is some simple matplotlib code to get you started.

In [ ]:
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
from captum.insights import AttributionVisualizer, Batch
from matplotlib.pyplot import imshow
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from PIL import Image




In [ ]:
fig, ax = plt.subplots(5,2, figsize=(20,20))

for i, (img, label) in enumerate(test_set):
  if i >= 10:
    break
  else:
    img = img.squeeze(dim=0)
    ax[i//2,i%2].title.set_text("Metastases" if label == 1 else "Benign")
    ax[i//2,i%2].imshow(img.permute(1,2,0))

### Great! Now you have trained and saved a model. Now we can begin answering some questions! This is where you will have to do some coding.

## 3.1 How explainable are our results? 
Run the code below to pick an image from 'test_set' and run integrated gradients. Use the output to explain whether or not the model is really looking for characteristics of metastasis. Report in your results whether or not the integrated gradients tell a meaningful story about the image. We do not expect a medical answer here, any form of reasoning is fine.

In [ ]:
"""
If you run this and get a GPU out of memory error. Restart the kernal and 
reload the data and reload your saved model. The training caused the GPU
memory to get filled up.

"""

def visualize_ig(model, img, label, baseline):
  img = img.unsqueeze(0).to(device)
  outputs = model(img)
  print(f"Raw model Output {outputs.item()}")
  outputs = nn.functional.softmax(outputs).reshape(-1)
  print(f"Model output after running through softmax {outputs.item()}")
  integrated_gradients = IntegratedGradients(model)

  attributions_ig = integrated_gradients.attribute(img, target=0)

  default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                  [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)
  noise_tunnel = NoiseTunnel(integrated_gradients)
  target = int(label)
  attributions_ig_nt = noise_tunnel.attribute(img, nt_samples=10, nt_type='smoothgrad_sq')
  _ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map"],
                                      ["all", "positive"],
                                      cmap=default_cmap,
                                        show_colorbar=True)


In [ ]:
pick = 0
sample_image = test_split[pick][0]
sample_label = test_split[pick][1]
visualize_ig(model, sample_image, sample_label, sample_image * 0)

### 3.2 How does occlusion effect our results?

Write a function `occlude` that takes in a image and randomly places a grey square that obstructs a part of the image. Then run the code to re-run the classifier on the occluded image comparing the model and integrated gradient output of the occluded verses the non-occluded image. 

In [ ]:
def occlude(img):
  #Your code goes here
  return 0

#Code below picks a random image, occludes, and shows the occluded image

#Modify pick to run around 5 images to get an idea for the effect
pick = 10 #Pick any number between 0 and 100 until you get an idea

sample_image = test_split[pick][0]
sample_label = test_split[pick][1]
sample_image_occluded = occlude(torch.clone(sample_image))
fig, ax = plt.subplots(1,1, figsize=(20,20))
sample_image_occluded.squeeze(dim=0)


ax.title.set_text("Metastases" if sample_label == 1 else "Benign")
ax.imshow(sample_image_occluded.permute(1,2,0))

In [ ]:
print("Results for the non-occluded image")
visualize_ig(model, sample_image, sample_label, sample_image * 0)

print("Results for the occluded image")
visualize_ig(model, sample_image_occluded, sample_label, sample_image_occluded * 0)

### 3.3 How can data augmentation help our model?

Our model above might be accurate, but it may or may not have done a good job explaining what metastasis looks like. During the training of our model we rescaled the images as a transformation. There are more transformations that we can do. Fill in `custom_transform` to include a set of other transformations. Then run the code to retrain the model with custom transform and see if your model begins to look for different structures using IG. Refer to the following: https://pytorch.org/docs/stable/torchvision/transforms.html for available transformations. Additionally look at `initial_transform` to see how to compose said transformations.

In [ ]:
def custom_transform(tensor, label):
  """
  Fill in this function with relevant code. 
  Think of using transformations like Horizontal Flip or Random Perspective
  Remember to first convert the image from np array to PILImage and then resize
  to 224 by 224 as this is required by most vision models.
  After transformations convert from a PIL image to tensor.
  Look at the link provided in the directions for more help.
 
  """ 
  return 0

pcam = PCam(transform=custom_transform)

sz = len(pcam)
size_train = math.floor(sz - (sz * (1 - PER_TRAIN)))
size_test = sz - size_train

print(f"Training size: {size_train}")
print(f"Testing size: {size_test}")

train_split, test_split = torch.utils.data.random_split(pcam, [size_train, size_test])
train_set = DataLoader(train_split, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=1)
test_set = DataLoader(test_split, batch_size=1,
                        shuffle=True, num_workers=1)


In [ ]:
#Training again, read earlier code about how training works 
model = train(train_set)

In [ ]:
#Saving model again but with different name this time
model_save_name = 'pcam_model_mod_transforms'
path = F"/content/gdrive/My Drive/BMI702/{model_save_name}" 
torch.save(model, path)

In [ ]:
#Loading model (code to remount if run into memory issues and have to restart)
from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'pcam_model_mod_transforms'
path = F"/content/gdrive/My Drive/BMI702/{model_save_name}" 
model = torch.load(path)

In [ ]:
"""
Here you can pick random images, the 0th image is picked right now, to view
how IG differs now that you have applied the transform.

"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pick = 0
sample_image = test_split[pick][0]
sample_label = test_split[pick][1]
visualize_ig(model, sample_image, sample_label, sample_image * 0)

### 3.4 How does noise effect our model?

Fill in the function `noise` to transform an input image by randomly adding noise. Note that the image should almost look identical to the human eye. Run the rest of the code to (1) run the image through the `noise` function and pass it to the classifier, (2) compute its IG, and (3) compare its IG to a non-transformed image. Please note whether or not the performance of your algorithm changes.

In [ ]:
def noise(tensor):
  #Fill in code here to add noise to an input tensor
  return 0

"""
After filling in the code above, the code below will show the image
with added noise in it. It should look like the normal image but with
small colored pixel dots everywhere. 

"""

pick = 0
sample_image = test_split[pick][0]
sample_label = test_split[pick][1]
sample_image_noise = noise(sample_image)
sample_image1 = sample_image_noise.unsqueeze(0).to('cpu')

fig, ax = plt.subplots(1,1, figsize=(20,20))
sample_image_noise.squeeze(dim=0)

ax.title.set_text("Metastases" if sample_label == 1 else "Benign")
ax.imshow(sample_image_noise.permute(1,2,0))

In [ ]:
"""
Run the code below to compare model output and IG for model with and without 
noise added. 
"""

print("Results for the regular image")
visualize_ig(model, sample_image, sample_label, sample_image * 0)

print("Results for the noise-added image")
visualize_ig(model, sample_image_noise, sample_label, sample_image_noise * 0)